In [ ]:
print('Lectura users')

path_users = list_objects_function(buckets_ = datalake_prod_bucket, 
                                   keys_ = gp1010_key, 
                                   retrieve_date = '', 
                                   date_logic = 'all')

columnas_usr = ['numero_cuenta', 'estado_cuenta', 'fecha_alta', 'fecha_nacimiento', 'sexo', 'provincia']
df_usr = spark.read.parquet(*path_users).select(columnas_usr)

#print(f'Hay {df_usr.count()} users')
df_usr = df_usr.dropDuplicates(['numero_cuenta'])
#print(f'Hay {df_usr.count()} users luego de sacar duplicados')

# dtypes
df_usr = (df_usr
          .withColumn('numero_cuenta', df_usr.numero_cuenta.cast('string'))
          .withColumn('fecha_alta', F.to_date(df_usr.fecha_alta,"yyyy-MM-dd"))
          .withColumn('fecha_nacimiento', F.to_date(df_usr.fecha_nacimiento,"yyyy-MM-dd"))
          .withColumn('provincia', df_usr.provincia.cast('int'))
         )

# Edad y antiguedad
df_usr = (df_usr
          .withColumn('edad', F.floor(F.datediff(F.current_date(), F.col('fecha_nacimiento'))/365.25))
          .withColumn('antiguedad', F.datediff(F.current_date(), F.col('fecha_alta')))
         )
          

          
df_usr = df_usr.withColumnRenamed('numero_cuenta','externalid')

# Provincia int a str
prov_dict = {
    1: 'CAPITAL_FEDERAL',
    2: 'BUENOS_AIRES',  # GRAN BUENOS AIRES
    3: 'BUENOS_AIRES',
    4: 'OTROS',  # CATAMARCA
    5: 'CORDOBA',
    6: 'NORTE_ESTE',  # CORRIENTES
    7: 'NORTE_ESTE',  # CHACO
    8: 'OTROS',  # CHUBUT
    9: 'OTROS',  # ENTRE RIOS
    10: 'NORTE_ESTE',  # FORMOSA
    11: 'NORTE',  # JUJUY
    12: 'OTROS',  # LA PAMPA
    13: 'OTROS',  # LA RIOJA
    14: 'MENDOZA',
    15: 'NORTE_ESTE',  # MISIONES
    16: 'OTROS',  # NEUQUEN
    17: 'OTROS',  # RIO NEGRO
    18: 'NORTE',  # SALTA
    19: 'OTROS',  # SAN JUAN
    20: 'OTROS',  # SAN LUIS
    21: 'SANTA_FE',
    22: 'OTROS',  # SANTA CRUZ
    23: 'OTROS',  # SANTIAGO DEL ESTERO
    24: 'OTROS',  # TIERRA DEL FUEGO
    25: 'NORTE'  # TUCUMAN
}

prov_mapping = F.create_map([F.lit(x) for x in chain(*prov_dict.items())])
df_usr = df_usr.withColumn('provincia', prov_mapping[df_usr['provincia']])

print('Unión con mappeo id')

df_usr = df_usr.join(df_mapeo_id_gp, df_usr.externalid == df_mapeo_id_gp.ext_id_mappeo, how='left')
df_usr = df_usr.drop('ext_id_mappeo')